# Portland Pizza Tour

## data clean up

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../scores/Portland Pizza Tour.csv")
df.head()

,Unnamed: 0,Pizza,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability (did they have what you wanted),sides,leftovers,total,Notes
0,Alex,Handsome Pizza,marinara,8.0,7.0,8.0,8.5,6.0,6.0,10.0,7.0,5.0,5.0,7.05,"not the best for vegan options. very tangy, bu..."
1,Alice,NaN,sergio olmos,7.0,7.0,8.0,NaN,8.0,8.0,9.0,7.0,5.0,3.0,6.89,no comment I guess
2,Alex,Mississippi pizza,amaizeing,8.0,8.0,7.0,6.0,9.0,9.0,NaN,9.0,8.0,8.0,8,great vegan options. caramelized onion sauce w...
3,Alice,NaN,spicoli,7.0,8.0,8.0,6.0,8.0,7.0,NaN,9.0,9.0,7.0,7.666666667,the salad was the highlight of this meal. it h...
4,Alex,Gracie's Apizza,vegan,9.0,9.0,9.0,9.0,9.0,9.0,NaN,9.0,6.0,NaN,8.625,a lot of things are right about this pizza. th...


In [3]:
import numpy as np

df.rename(
    columns={
        "Unnamed: 0": "Name",
        "Pizza": "Vendor",
        "Availability (did they have what you wanted)": "Availability",
    },
    inplace=True,
)
df.total.replace("#DIV/0!", np.nan, inplace=True)

In [4]:
df.Vendor.fillna(method="ffill", inplace=True)
df.drop_duplicates(subset=["Name", "Vendor"], inplace=True)
df.head()

,Name,Vendor,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,total,Notes
0,Alex,Handsome Pizza,marinara,8.0,7.0,8.0,8.5,6.0,6.0,10.0,7.0,5.0,5.0,7.05,"not the best for vegan options. very tangy, bu..."
1,Alice,Handsome Pizza,sergio olmos,7.0,7.0,8.0,NaN,8.0,8.0,9.0,7.0,5.0,3.0,6.89,no comment I guess
2,Alex,Mississippi pizza,amaizeing,8.0,8.0,7.0,6.0,9.0,9.0,NaN,9.0,8.0,8.0,8,great vegan options. caramelized onion sauce w...
3,Alice,Mississippi pizza,spicoli,7.0,8.0,8.0,6.0,8.0,7.0,NaN,9.0,9.0,7.0,7.666666667,the salad was the highlight of this meal. it h...
4,Alex,Gracie's Apizza,vegan,9.0,9.0,9.0,9.0,9.0,9.0,NaN,9.0,6.0,NaN,8.625,a lot of things are right about this pizza. th...


In [5]:
df.drop(df[df.total.isna()].index, inplace=True)
df.shape

(46, 15)

In [6]:
df["Name"].value_counts()

Name
Alex     24
Alice    22
Name: count, dtype: int64

In [7]:
df.isna().sum()

Name             0
Vendor           0
Variety          5
Taste            0
Presentation     0
Crust            0
Sauce            1
Toppings         0
Creativity       0
Message         32
Availability     0
sides           30
leftovers       32
total            0
Notes           21
dtype: int64

In [8]:
df.columns

Index(['Name', 'Vendor', 'Variety', 'Taste', 'Presentation', 'Crust', 'Sauce',
       'Toppings', 'Creativity', 'Message', 'Availability', 'sides',
       'leftovers', 'total', 'Notes'],
      dtype='object')

In [9]:
score_cols = [
    "Taste",
    "Presentation",
    "Crust",
    "Sauce",
    "Toppings",
    "Creativity",
    "Message",
    "Availability",
    "sides",
    "leftovers",
]
len(score_cols)

10

In [10]:
# what is the maximum number of missing score values in a row?
df[score_cols].T.isna().sum().max()

3

currenty rankings with NaN values in the score are not being counted towards the total score, so more NaN values inflates the score. Need to correct for this. Instead we'll take the top 4 scores combined with the bottom 3 scores (total of 7 so in the worst case all scores will be populated) and average those

In [11]:
def get_final_score(row, top_k=4):
    """Calculate the final score for a row of pizza scores."""
    scores = row.dropna().sort_values(ascending=False).values
    return np.mean(np.concatenate([scores[top_k:], scores[(7 - top_k) * -1 :]]))

In [12]:
final_scores = []
for _, row in df[score_cols].iterrows():
    final_scores.append(get_final_score(row))

df["final_score"] = final_scores

In [13]:
df.head()

,Name,Vendor,Variety,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,total,Notes,final_score
0,Alex,Handsome Pizza,marinara,8.0,7.0,8.0,8.5,6.0,6.0,10.0,7.0,5.0,5.0,7.05,"not the best for vegan options. very tangy, bu...",5.777778
1,Alice,Handsome Pizza,sergio olmos,7.0,7.0,8.0,NaN,8.0,8.0,9.0,7.0,5.0,3.0,6.89,no comment I guess,5.500000
2,Alex,Mississippi pizza,amaizeing,8.0,8.0,7.0,6.0,9.0,9.0,NaN,9.0,8.0,8.0,8,great vegan options. caramelized onion sauce w...,7.250000
3,Alice,Mississippi pizza,spicoli,7.0,8.0,8.0,6.0,8.0,7.0,NaN,9.0,9.0,7.0,7.666666667,the salad was the highlight of this meal. it h...,6.875000
4,Alex,Gracie's Apizza,vegan,9.0,9.0,9.0,9.0,9.0,9.0,NaN,9.0,6.0,NaN,8.625,a lot of things are right about this pizza. th...,8.142857


In [14]:
# sort by person then by final score
df.sort_values(["Name", "final_score"], ascending=False, inplace=True)

In [15]:
df.drop(columns=["total"], inplace=True)
df = df[["Name", "Vendor", "final_score"] + score_cols + ["Notes"]]

In [16]:
alex = df[df["Name"] == "Alex"].copy()
alice = df[df["Name"] == "Alice"].copy()
alex.drop(columns=["Name"], inplace=True)
alice.drop(columns=["Name"], inplace=True)

## Top 5 for Vegan Options

In [17]:
alex.head()

,Vendor,final_score,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,Notes
18,Pizzeria Otto,9.142857,10.0,10.0,10.0,10.0,10.0,9.0,9.0,9.0,NaN,NaN,the vegan cashew ricotta is the best I've ever...
28,Pizza Doughnais (sourdough),8.500000,10.0,8.0,10.0,10.0,9.0,9.0,8.0,9.0,NaN,9.0,NaN
14,Red Sauce Pizza,8.428571,10.0,8.0,9.0,9.0,10.0,8.0,9.0,9.0,NaN,NaN,this pizza really hit the spot. a classic comb...
26,Scottie's Pizza Parlor,8.428571,10.0,10.0,10.0,10.0,8.0,8.0,NaN,9.0,NaN,9.0,NaN
50,Fino Bistro,8.250000,10.0,9.0,9.0,10.0,10.0,9.0,NaN,8.0,8.0,8.0,Incredible roasted red pepper base!


## Top 5 for meat eaters

In [18]:
alice.head()

,Vendor,final_score,Taste,Presentation,Crust,Sauce,Toppings,Creativity,Message,Availability,sides,leftovers,Notes
19,Pizzeria Otto,8.857143,10.0,10.0,8.0,10.0,10.0,9.0,9.0,10.0,NaN,NaN,The inner crust is the shining star. I have ne...
37,Dove Vivi,8.666667,10.0,9.0,10.0,8.0,10.0,9.0,NaN,10.0,NaN,NaN,very very interesting. only complaint is that ...
7,Ranch Pizza,8.142857,9.0,9.0,10.0,8.0,8.0,8.0,NaN,9.0,NaN,9.0,The pan style crust blew me away. reminded me ...
11,Lovelysfiftyfifty,8.142857,10.0,9.0,9.0,9.0,9.0,10.0,NaN,8.0,7.0,NaN,"small selection, however they did have everyth..."
9,Tastebud,8.000000,9.0,7.0,8.0,9.0,8.0,9.0,9.0,8.0,9.0,9.0,"I normally really enjoy this place, but today ..."


In [21]:
alex.to_csv("../scores/vegan.csv", index=False)
alice.to_csv("../scores/non-vegan.csv", index=False)